Scoring goals is the hardest thing to do in the game of football. This is why in this analysis I will check, if forwards, whose main role is to score goals, actually score more goals than players in other positions. Data containes number of goals scored by all players in 5 best european leagues. (Spanish, French, German, English and Italian first divisions)  <br>
The analysis is based on data from https://fbref.com, from which I have downloaded player data for 2020/2021 season.

In [ ]:
# Importing libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In order to make the dataset, firstly I have downloaded 5 tables with all players stats from top 5 leagues (English, Spanish, German, French and Italian first divisions) onto my computer. Secondly, I have merged them, and deleted all columns except for name, position and number of goals scored.

In [ ]:
df = pd.read_csv('../input/20202021-player-stats/2020-2021 player stats.csv')
del df['Unnamed: 0']
df.head()

Next step is to clean the data in order for it to properly work.

In [ ]:
# Deleting NaN rows (with player name being 'Player') and checking for duplicates
players = df['1_Player']
df = df[players != 'Player']
players.duplicated(keep='first').sum()

In [ ]:
df.dtypes

In [ ]:
#Converting goals column into integers
df['Performance_Gls'] = df['Performance_Gls'].astype('int64')
df.dtypes

In [ ]:
# Merging duplicated rows (for players playing in multiple teams during a season)
df['Performance_Gls'] = df.groupby('1_Player')['Performance_Gls'].transform('sum')
df = df.drop_duplicates(subset='1_Player')

In [ ]:
df.shape

In [ ]:
df.describe()

From the describe method we can see, that 50% of all players didn't score a single goal during the whole season. <br>
Let's see the distribution of goals scored.

In [ ]:
# Defining ECDF function to visualise distribution
def ecdf(data):
    # n - number of data points
    n = len(data)
    # x - x-data for ECDF
    x = np.sort(data)
    # y - y-data for ECDF
    y = np.arange(1, n+1) / n
    return x, y

In [ ]:
# PDF of goals scored by all players during 2020/2021 season
plt.hist(df['Performance_Gls'], bins=5, density=True)
plt.xlabel('Number of goals scored')
plt.ylabel('PDF')
plt.show()

As shown above, most of the players have scored less than 10 goals. Now let's conduct an ECDF

In [ ]:
# Computing ECDF for amount of goals scored by player during 2020/2021 season
#n - number of data points
x, y = ecdf(df['Performance_Gls'])
#Generate ECDF plot
plt.scatter(x, y)
plt.xlabel('Number of goals scored')
plt.ylabel('ECDF')
plt.show()

Almost all of the players scored 10 goals of less, and 50% of all players didn't score a single goal. But since dataset contains all players, I have decided to exclude goalkeepers from the data, because all of 205 of them managed to score only 3 goals combined.

In [ ]:
goals_by_goalkeepers = df[df['3_Pos'] == 'GK']['Performance_Gls'].sum()
print(f'Combined goals by goalkeepers: {goals_by_goalkeepers}')

In [ ]:
df['3_Pos'].value_counts()

In [ ]:
df = df[df['3_Pos'] != 'GK']
df = df[df['3_Pos'] != 'GK,MF']
df['3_Pos'].value_counts()

In order to answear the main question, firstly I will compare distributions of goals scored by forwards and players in different possitions.

In [ ]:
fw_cols = ['FW','FW,MF']
forwards = df.loc[df['3_Pos'].isin(fw_cols)]

rest_cols = ['DF','MF','MF,FW','DF,MF','MF,DF','FW,DF']
rest = df.loc[df['3_Pos'].isin(rest_cols)]

x_fw, y_fw = ecdf(forwards['Performance_Gls'])
plt.scatter(x_fw, y_fw, label='Forwards')

x_rest, y_rest = ecdf(rest['Performance_Gls'])
plt.scatter(x_rest, y_rest, label='Other positions (excluding goalkeepers)')

plt.xlabel('Number of goals scored by a player')
plt.ylabel('ECDF')
plt.legend()
plt.show()

Eyeballing the data suggest that forwards actually do score a lot more goals. <br>
Now let's estimate parameters before performing hypothesis test.

In [ ]:
# Difference of means between empirical data
mean_diff = forwards['Performance_Gls'].mean() - rest['Performance_Gls'].mean()
print(f'Difference of means = {mean_diff}')

In order to test the hypothesis, I will simulate 2020/2021 season 10 000 times in order to get a big sample of permutated data.<br> It will allow to test the null hyphotesis, which states that difference between mean number of goals scored by forwards and players in other possitions is statistically insignificant.

In [ ]:
# Mean for bootstrap replicates for forwards and the rest
bs_rep_fw = np.empty(10000)
for i in range(10000):
    bs_rep_fw[i] = np.mean(np.random.choice(forwards['Performance_Gls'], size=len(forwards)))

bs_rep_rest = np.empty(10000)
for i in range(10000):
    bs_rep_rest[i] = np.mean(np.random.choice(rest['Performance_Gls'], size=len(rest)))

In [ ]:
# Samples of difference of means
bs_rep_mean_diff = bs_rep_fw - bs_rep_rest

In [ ]:
# 95% confidence interval for difference of replicate means 
conf_int = np.percentile(bs_rep_mean_diff, [2.5, 97.5])
print(f'95% confidence interval = {conf_int}')

In [ ]:
# Calculating p value
p = np.sum(bs_rep_mean_diff >= mean_diff) / len(bs_rep_mean_diff)
print(f'p value = {p}')

As p value is very high, we should conclude that forwards do actually score more goals than players in other positions.